# Senior Design Team 1725, Classifying AIS, Alternative Model Training
Yolov8 Testing- Ziek

This file is a juptyer notebook containing the training results using Yolov8 as a alternative solution model while Joey and Landon work with Yolov11

## Dataset Mounting (Sharepoint):
Before getting to the model we first need to access the dataset. This is done using our sharepoint/OneDrive to colab.

Currently rlcone config must be first run *locally* then once obtaining the API key it can be run on colab. This is so that the mount works properly, which requires web browser access (this is not possible in colab)

In [1]:
!wget https://downloads.rclone.org/v1.69.0/rclone-v1.69.0-linux-amd64.deb
!apt install ./rclone-v1.69.0-linux-amd64.deb

--2025-02-21 00:46:30--  https://downloads.rclone.org/v1.69.0/rclone-v1.69.0-linux-amd64.deb
Resolving downloads.rclone.org (downloads.rclone.org)... 95.217.6.16, 2a01:4f9:c012:7154::1
Connecting to downloads.rclone.org (downloads.rclone.org)|95.217.6.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23665802 (23M) [application/vnd.debian.binary-package]
Saving to: ‘rclone-v1.69.0-linux-amd64.deb’

rclone-v1.69.0-linu 100%[===================>]  22.57M  13.7MB/s    in 1.7s    

2025-02-21 00:46:32 (13.7 MB/s) - ‘rclone-v1.69.0-linux-amd64.deb’ saved [23665802/23665802]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'rclone' instead of './rclone-v1.69.0-linux-amd64.deb'
The following NEW packages will be installed:
  rclone
0 upgraded, 1 newly installed, 0 to remove and 21 not upgraded.
Need to get 0 B/23.7 MB of archives.
After this operation, 71.0 MB of additional disk space will be used.
Get

In [6]:
# We will need to pass some arguments for this to work,
# so make sure we still have the api key
# (Note: you need to copy the ENTIRE struct, as the values are read as an array)

# Also, make sure to clear the output! We don't want to have
# loose api keys floating around the internet!
!rclone config

#current API key (update this regularly):
#website link: https://pennstateoffice365.sharepoint.com/sites/AISSeniorDesignProject

Current remotes:

Name                 Type
====                 ====
onedrive             onedrive

e) Edit existing remote
n) New remote
d) Delete remote
r) Rename remote
c) Copy remote
s) Set configuration password
q) Quit config
e/n/d/r/c/s/q> d

Select remote.
Choose a number from below, or type in an existing value.
 1 > onedrive
remote> 1


No remotes found, make a new one?
n) New remote
s) Set configuration password
q) Quit config
n/s/q> n

Enter name for new remote.
name> onedrive

Option Storage.
Type of storage to configure.
Choose a number from below, or type in your own value.
 1 / 1Fichier
   \ (fichier)
 2 / Akamai NetStorage
   \ (netstorage)
 3 / Alias for an existing remote
   \ (alias)
 4 / Amazon S3 Compliant Storage Providers including AWS, Alibaba, ArvanCloud, Ceph, ChinaMobile, Cloudflare, DigitalOcean, Dreamhost, GCS, HuaweiOBS, IBMCOS, IDrive, IONOS, LyveCloud, Leviia, Liara, Linode, Magalu, Minio, Netease, Outscale, Petabox, RackCorp, Rclone, Scaleway, Seaweed

In [7]:
!sudo mkdir /content/onedrive
# Create onedrive directory in colab

mkdir: cannot create directory ‘/content/onedrive’: File exists


In [8]:
# Install fusermount3 to ensure proper mounting
!sudo apt-get -y install fuse3

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following package was automatically installed and is no longer required:
  libfuse2
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  libfuse3-3
The following packages will be REMOVED:
  fuse
The following NEW packages will be installed:
  fuse3 libfuse3-3
0 upgraded, 2 newly installed, 1 to remove and 21 not upgraded.
Need to get 106 kB of archives.
After this operation, 264 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libfuse3-3 amd64 3.10.5-1build1 [81.2 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 fuse3 amd64 3.10.5-1build1 [24.7 kB]
Fetched 106 kB in 0s (441 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 

In [9]:
!nohup rclone --vfs-cache-mode full mount onedrive: /content/onedrive &
# After these commands, we should have a onedrive folder in the files section
# within colab with everything we need!

nohup: appending output to 'nohup.out'


## Dataset Setup:

Now that the dataset is mount we need to copy the files from the datasets taht we want to use for creating our model

Note: This will consume a lot of disk resources on Colab.

In [10]:
# Import utilities to take random image sampling and copy to target directory
import os
import shutil
import random

# Create a new directory specifically to hold dataset
!sudo mkdir /content/SenDsgnTraining

In [11]:
# Code assistance from Joey George
# Dataset path:
source_dir = r"/content/onedrive/Training Images/Training"
target_dir = r"/content/SenDsgnTraining"

# Define classes for the smaller datasets
# The options we have are:
# "narrowleaf_cattail", "none", "phragmites", "purple_loosestrife"
classes = ["none", "purple_loosestrife", "phragmites", "narrowleaf_cattail"]
splits = {"train": 70, "val": 15, "test": 15}  # Percentage split

In [12]:
# Create target directory structure
for split in splits:
    for class_name in classes:
        os.makedirs(os.path.join(target_dir, split, class_name), exist_ok=True)

In [13]:
# Process each class
for class_name in classes:
    source_path = os.path.join(source_dir, class_name)
    images = [f for f in os.listdir(source_path)] #checking for .jpg isn't needed as some images aren't .jpg

    # Randomly sample 1500 images
    selected_images = random.sample(images, min(1500, len(images)))

    # Distribute images across train, val, test
    split_counts = {k: int(v * len(selected_images) / 100) for k, v in splits.items()}
    start_idx = 0
    for split, count in split_counts.items():
        split_path = os.path.join(target_dir, split, class_name)
        for i in range(count):
            img_name = f"{start_idx}_{class_name}.jpg"
            shutil.copy(os.path.join(source_path, selected_images[start_idx]), os.path.join(split_path, img_name))
            start_idx += 1

##Training

With the dataset properly taken care of through sharepoint method, the training can begin.

The best thing about the classification model is that no .yaml file is necessary, as the folder structure is enough to ensure the training, validation, and testing splits are recognized for each class.

In [14]:
# Install Ultralytics and any required dependencies
%pip install ultralytics  # install
from ultralytics import YOLO, checks
checks()  # checks

Ultralytics 8.3.78 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Setup complete ✅ (12 CPUs, 83.5 GB RAM, 129.5/235.7 GB disk)


In [15]:
# Load the pretrained Yolov8 Model
model = YOLO("yolov8n-cls.pt")

100%|██████████| 5.31M/5.31M [00:00<00:00, 63.1MB/s]


In [16]:
# Train the model
results = model.train(data=target_dir, epochs=50, imgsz=640, batch=16)

Ultralytics 8.3.78 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: task=classify, mode=train, model=yolov8n-cls.pt, data=/content/SenDsgnTraining, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf

100%|██████████| 5.35M/5.35M [00:00<00:00, 66.5MB/s]


AMP: checks passed ✅


train: Scanning /content/SenDsgnTraining/train... 3306 images, 0 corrupt: 100%|██████████| 3306/3306 [00:07<00:00, 444.23it/s]


train: New cache created: /content/SenDsgnTraining/train.cache


...

      50/50      1.54G    0.01203         10        640: 100%|██████████| 207/207 [02:47<00:00,  1.24it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 23/23 [00:18<00:00,  1.24it/s]

                   all      0.987          1



50 epochs completed in 2.554 hours.
Optimizer stripped from runs/classify/train/weights/last.pt, 3.0MB
Optimizer stripped from runs/classify/train/weights/best.pt, 3.0MB

Validating runs/classify/train/weights/best.pt...
Ultralytics 8.3.78 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
YOLOv8n-cls summary (fused): 30 layers, 1,440,004 parameters, 0 gradients, 3.3 GFLOPs
train: /content/SenDsgnTraining/train... found 3306 images in 4 classes ✅ 
val: /content/SenDsgnTraining/val... found 708 images in 4 classes ✅ 
test: /content/SenDsgnTraining/test... found 708 images in 4 classes ✅ 


               classes   top1_acc   top5_acc: 100%|██████████| 23/23 [00:40<00:00,  1.77s/it]


                   all      0.989          1
Speed: 0.4ms preprocess, 0.3ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train


In [17]:
# For further uses where we want to download the "runs" folder and save the training results
!zip -r /content/run2.zip /content/runs

  adding: content/runs/ (stored 0%)
  adding: content/runs/classify/ (stored 0%)
  adding: content/runs/classify/train/ (stored 0%)
  adding: content/runs/classify/train/train_batch1.jpg (deflated 1%)
  adding: content/runs/classify/train/results.csv (deflated 68%)
  adding: content/runs/classify/train/train_batch8280.jpg (deflated 0%)
  adding: content/runs/classify/train/results.png (deflated 10%)
  adding: content/runs/classify/train/weights/ (stored 0%)
  adding: content/runs/classify/train/weights/best.pt (deflated 9%)
  adding: content/runs/classify/train/weights/last.pt (deflated 9%)
  adding: content/runs/classify/train/val_batch0_pred.jpg (deflated 0%)
  adding: content/runs/classify/train/confusion_matrix_normalized.png (deflated 26%)
  adding: content/runs/classify/train/val_batch2_labels.jpg (deflated 0%)
  adding: content/runs/classify/train/val_batch1_pred.jpg (deflated 0%)
  adding: content/runs/classify/train/train_batch0.jpg (deflated 1%)
  adding: content/runs/classif